In [17]:
import numpy as np
import pandas as pd
from gensim.models import word2vec 
import gensim

In [18]:
def read_data(filename):
    data = []
    with open(filename, 'r', encoding='utf-8') as myfile:
        f = myfile.readlines()
        s_num = 0
        i =0
        sentence_s = []
        tag_s = []
        dep_s = []
        s  = []   # sentence
        p = []    # tag
        d = []    # dependency
        for l in f:
            
            v = l.replace('\n','').split("\t")
            v.append(s_num)
            if len(l) != 1:
                data.append(v)
                dep = v[6] + '_' + v[7]
                word = v[1].lower()
                if any(char.isdigit() for char in word):
                    word = 'NUM'       # replace numbers with NUM
                s.append(word)
                p.append(v[3])
                d.append(dep)
                i +=1
            else:
                sentence_s.append(s)
                tag_s.append(p)
                dep_s.append(d)
                s_num +=1
                s  = []
                p = []
                d = []
        
    return data, sentence_s, tag_s, dep_s

In [19]:
train_data, train_sentences, train_tags, train_dependencies = read_data('./data/train-stanford-raw.conll')
dev_data, dev_sentences, dev_tags, dev_dependencies = read_data('./data/dev-stanford-raw.conll')
test_data, test_sentences, test_tags, test_dependencies = read_data('./data/test-stanford-raw.conll')

# create a full set of all the words in our train, test, and dev sets for word2vec model
# in order to avoid unseen words during test and validation
total_sentences = train_sentences + dev_sentences + test_sentences
print('data: ', data[:5])
print('words sentences: ', total_sentences[2:5])
print('tags sentences: ', tags[2:5])
print('dependencies: ', dependencies[2:5])

data:  [['1', 'In', '_', 'IN', 'IN', '_', '45', 'prep', '_', '_', 0], ['2', 'an', '_', 'DT', 'DT', '_', '5', 'det', '_', '_', 0], ['3', 'Oct.', '_', 'NNP', 'NNP', '_', '5', 'nn', '_', '_', 0], ['4', '19', '_', 'CD', 'CD', '_', '5', 'num', '_', '_', 0], ['5', 'review', '_', 'NN', 'NN', '_', '1', 'pobj', '_', '_', 0]]
words sentences:  [['rolls-royce', 'motor', 'cars', 'inc.', 'said', 'it', 'expects', 'its', 'u.s.', 'sales', 'to', 'remain', 'steady', 'at', 'about', 'NUM', 'cars', 'in', 'NUM', '.'], ['the', 'luxury', 'auto', 'maker', 'last', 'year', 'sold', 'NUM', 'cars', 'in', 'the', 'u.s.'], ['howard', 'mosher', ',', 'president', 'and', 'chief', 'executive', 'officer', ',', 'said', 'he', 'anticipates', 'growth', 'for', 'the', 'luxury', 'auto', 'maker', 'in', 'britain', 'and', 'europe', ',', 'and', 'in', 'far', 'eastern', 'markets', '.']]
tags sentences:  [['NNP', 'NNP', 'NNPS', 'NNP', 'VBD', 'PRP', 'VBZ', 'PRP$', 'NNP', 'NNS', 'TO', 'VB', 'JJ', 'IN', 'IN', 'CD', 'NNS', 'IN', 'CD', '.'],

In [20]:
model_name = "dep_parser_word2vec_total"

# train it:
# print("training word2vec model")
# model = word2vec.Word2Vec(total_sentences,size=189, min_count=1) # here moeten we wat tweaken met de parameters
# print("model trained")
# model.save(model_name)
# print("model saved")

# or load it:
model = Word2Vec.load(model_name)
print("model loaded from disk")

training word2vec model
model trained
model saved


In [23]:
vocab = model.vocab.keys()
print("V (length of training vocabulary): " + str(len(vocab)))
print("most similar positive test: ")
print(model.most_similar(positive=['woman', 'king'], negative=['man']))
print("does not match test: ")
print(model.doesnt_match("breakfast cereal dinner lunch".split()))
print("word similarity test: ")
print(model.similarity('woman', 'man'))
print()
print(model['computer'])  # raw numpy vector of a word

V (length of training vocabulary): 35532
[('beam', 0.9523080587387085), ("'n'", 0.9470729231834412), ('barry', 0.9401122331619263), ('marlboro', 0.939976155757904), ('aloft', 0.9398140907287598), ('colo.', 0.9389971494674683), ('manufacturer', 0.9372619986534119), ('constructed', 0.9367369413375854), ('angels', 0.9350725412368774), ('glossy', 0.9349611401557922)]
cereal
0.954927562839
[ 0.03253329  0.29376748  0.42870235 -0.31171563 -0.44949016 -0.5051102
 -0.53029168 -0.36873311  0.06408304 -0.02383715  0.68958682  0.31396839
  0.30426395 -0.28738719  0.20859465  0.45424727 -0.11529829  0.0015334
 -0.04299233 -0.12471952 -0.52828872 -0.11824053  0.37391958  0.3078886
  0.39943048  0.42146483  0.50414109  0.21233779 -0.44621235  0.20821989
 -0.31146091 -0.1204848  -0.32732096 -0.11361341 -0.20991087 -0.51544493
 -0.07378293 -0.47551197  0.37150779 -0.22198907  0.6451869   0.25483611
  0.26936471 -0.11964172 -0.03437093 -0.1712622   0.34830585  0.14544176
 -0.18600805  0.32915574  0.651